<a href="https://colab.research.google.com/github/ghfri-code/Bayesian-Classification-oneGaussianDistribution/blob/master/semantic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

data = {
    "question": [
        "ساعات کاری فروشگاه شما چیست؟",
        "چطور می‌توانم سفارش را لغو کنم؟",
        "آیا امکان بازگشت کالا وجود دارد؟",
        "روش‌های پرداخت شما چیست؟"
    ],
    "answer": [
        "ساعات کاری ما همه روزه از ۹ صبح تا ۹ شب است.",
        "برای لغو سفارش به بخش سفارشات در حساب کاربری بروید.",
        "بله، تا ۷ روز امکان بازگشت کالا وجود دارد.",
        "پرداخت از طریق کارت بانکی و درگاه آنلاین امکان‌پذیر است."
    ]
}

df = pd.DataFrame(data)

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')
faq_embeddings = model.encode(df['question'].tolist(), convert_to_tensor=True)

# Clean and multi-purpose chatbot
def semantic_chatbot(query):
    # If input is a single string, convert it to a list
    single_input = False

    # Check if input is str or not
    if isinstance(query, str):
        query = [query]
        single_input = True

    # Compute embeddings and cosine similarity
    query_emb = model.encode(query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_emb, faq_embeddings)
    best_matches = torch.argmax(scores, dim=1)

    # Retrieve answers and remove zero-width non-joiner (ZWNJ)
    answers = [df['answer'].iloc[int(i)].replace("\u200c", "") for i in best_matches]

    # If input was a single sentence, return only one string
    if single_input:
        return answers[0]
    return answers


# Test

print("User: چطور سفارش رو لغو کنم؟")
print("Bot:", semantic_chatbot("چطور سفارش رو لغو کنم؟"))

queries = [
    "چطور سفارشم رو لغو کنم؟",
    "روش‌های پرداختتون چیه؟"
]

print(f"User: {"\n\t".join(queries)}")
print("Bot:", semantic_chatbot(queries))
